In [14]:
# TEST MARKDOWN EXTRACTION WITH OVERLAP - Single Cell
import fitz
import pymupdf4llm
from pathlib import Path
import yaml

# Load config to get overlap setting
with open("config/config.yaml", 'r') as f:
    config = yaml.safe_load(f)

max_pages = config['processing']['max_test_pages']
OVERLAP_SIZE = config['chunking']['overlap']  # ✅ Get overlap from config

# Pick a PDF to test
pdf_path = Path("test_dataset/markdown_extraction/1-s2.0-S0720048X23000712-main.pdf")

print("="*70)
print(f"📄 EXTRACTING: {pdf_path.name}")
print(f"   Method: MARKDOWN")
print(f"   Overlap: {OVERLAP_SIZE} chars")  # ✅ Show overlap setting
print("="*70)

doc = fitz.open(pdf_path)
total_pages = min(max_pages, doc.page_count)

print(f"📊 Total pages to process: {total_pages}")
print("="*70)

# ✅ NEW HELPER FUNCTION - Same as in extractors.py
def get_overlap(text: str, overlap_size: int) -> str:
    """Get the last N characters from text, trying to break at sentence boundary."""
    if len(text) <= overlap_size:
        return text
    
    overlap = text[-overlap_size:]
    
    sentence_start = overlap.find('. ')
    if sentence_start != -1 and sentence_start < overlap_size * 0.5:
        overlap = overlap[sentence_start + 2:]
    
    return overlap

all_chunks = []
MAX_SIZE = 950

# Process each page
for page_num in range(total_pages):
    print(f"\n   Page {page_num + 1}/{total_pages}")
    print(f"      Extracting text (markdown)...")
    
    try:
        md_dict = pymupdf4llm.to_markdown(str(pdf_path), pages=[page_num])
        
        if isinstance(md_dict, dict) and 'text' in md_dict:
            text = md_dict['text']
        else:
            text = md_dict
        
        if not text.strip():
            print(f"       ✅ 0 text chunks (empty page)")
            continue
        
        # ✅ UPDATED CHUNKING WITH OVERLAP
        page_chunks = []
        chunk_idx = 0
        
        paragraphs = [p.strip() for p in text.split('\n\n') if p.strip() and len(p.strip()) > 10]
        current_chunk = ""
        overlap_text = ""  # ✅ Track overlap
        
        for para in paragraphs:
            if len(para) > MAX_SIZE:
                if current_chunk.strip():
                    page_chunks.append({
                        'page': page_num + 1,
                        'chunk_idx': chunk_idx,
                        'text': current_chunk.strip()
                    })
                    chunk_idx += 1
                    # ✅ Create overlap
                    overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
                    current_chunk = overlap_text
                
                sentences = [s.strip() + '.' for s in para.split('.') if s.strip()]
                for sent in sentences:
                    if len(current_chunk) + len(sent) > MAX_SIZE:
                        if current_chunk.strip():
                            page_chunks.append({
                                'page': page_num + 1,
                                'chunk_idx': chunk_idx,
                                'text': current_chunk.strip()
                            })
                            chunk_idx += 1
                            # ✅ Create overlap
                            overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
                        current_chunk = overlap_text + sent + " "
                    else:
                        current_chunk += sent + " "
            
            elif len(current_chunk) + len(para) > MAX_SIZE:
                if current_chunk.strip():
                    page_chunks.append({
                        'page': page_num + 1,
                        'chunk_idx': chunk_idx,
                        'text': current_chunk.strip()
                    })
                    chunk_idx += 1
                    # ✅ Create overlap
                    overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
                current_chunk = overlap_text + para + "\n\n"
            else:
                current_chunk += para + "\n\n"
        
        # Save last chunk
        if current_chunk.strip():
            if len(current_chunk) > 1000:
                current_chunk = current_chunk[:1000]
            page_chunks.append({
                'page': page_num + 1,
                'chunk_idx': chunk_idx,
                'text': current_chunk.strip()
            })
        
        all_chunks.extend(page_chunks)
        print(f"       ✅ {len(page_chunks)} text chunks")
        
    except Exception as e:
        print(f"       ⚠️  Markdown extraction failed: {e}")

doc.close()

print(f"\n{'='*70}")
print(f"✅ EXTRACTION COMPLETE: {len(all_chunks)} total chunks")
print(f"{'='*70}")

# Print all chunks WITH OVERLAP HIGHLIGHTING
print(f"\n\n📄 ALL EXTRACTED CHUNKS (with overlap visualization):")
print("="*70)

for i, chunk in enumerate(all_chunks):
    print(f"\n{'='*70}")
    print(f"CHUNK {i+1} | Page {chunk['page']} | Chunk Index {chunk['chunk_idx']}")
    print(f"Length: {len(chunk['text'])} chars")
    
    # ✅ HIGHLIGHT OVERLAP with previous chunk
    if i > 0:
        prev_chunk = all_chunks[i-1]['text']
        curr_chunk = chunk['text']
        
        # Find overlap between previous and current
        overlap_found = None
        for overlap_len in range(min(len(prev_chunk), len(curr_chunk)), 20, -1):
            if prev_chunk[-overlap_len:] == curr_chunk[:overlap_len]:
                overlap_found = overlap_len
                break
        
        if overlap_found and overlap_found > 30:
            print(f"🔄 OVERLAP DETECTED: {overlap_found} chars")
            print(f"   Previous chunk ends with: ...{prev_chunk[-100:]}")
            print(f"   This chunk starts with: {curr_chunk[:100]}...")
    
    print(f"{'='*70}")
    print(chunk['text'])

print(f"\n\n{'='*70}")
print(f"📊 SUMMARY")
print(f"{'='*70}")
print(f"Total chunks extracted: {len(all_chunks)}")
print(f"Pages processed: {total_pages}")
print(f"Overlap setting: {OVERLAP_SIZE} chars")
print(f"Average chunk size: {sum(len(c['text']) for c in all_chunks) / len(all_chunks):.0f} chars")

📄 EXTRACTING: 1-s2.0-S0720048X23000712-main.pdf
   Method: MARKDOWN
   Overlap: 150 chars
📊 Total pages to process: 13

   Page 1/13
      Extracting text (markdown)...
       ✅ 8 text chunks

   Page 2/13
      Extracting text (markdown)...
       ✅ 7 text chunks

   Page 3/13
      Extracting text (markdown)...
       ✅ 4 text chunks

   Page 4/13
      Extracting text (markdown)...
       ✅ 9 text chunks

   Page 5/13
      Extracting text (markdown)...
       ✅ 9 text chunks

   Page 6/13
      Extracting text (markdown)...
       ✅ 8 text chunks

   Page 7/13
      Extracting text (markdown)...
       ✅ 7 text chunks

   Page 8/13
      Extracting text (markdown)...
       ✅ 7 text chunks

   Page 9/13
      Extracting text (markdown)...
       ✅ 18 text chunks

   Page 10/13
      Extracting text (markdown)...
       ✅ 10 text chunks

   Page 11/13
      Extracting text (markdown)...
       ✅ 5 text chunks

   Page 12/13
      Extracting text (markdown)...
       ✅ 24 text chunks

In [13]:
# # TEST RAW EXTRACTION WITH OVERLAP - Single Cell
# import fitz
# from pathlib import Path
# import yaml

# # Load config
# with open("config/config.yaml", 'r') as f:
#     config = yaml.safe_load(f)

# max_pages = config['processing']['max_test_pages']
# OVERLAP_SIZE = config['chunking']['overlap']

# # Pick a PDF to test (same one you used for markdown)
# pdf_path = Path("test_dataset/markdown_extraction/1-s2.0-S0720048X23000712-main.pdf")

# print("="*70)
# print(f"📄 EXTRACTING: {pdf_path.name}")
# print(f"   Method: RAW")
# print(f"   Overlap: {OVERLAP_SIZE} chars")
# print("="*70)

# doc = fitz.open(pdf_path)
# total_pages = min(max_pages, doc.page_count)

# print(f"📊 Total pages to process: {total_pages}")
# print("="*70)

# # Helper function
# def get_overlap(text: str, overlap_size: int) -> str:
#     """Get the last N characters from text, trying to break at sentence boundary."""
#     if len(text) <= overlap_size:
#         return text
    
#     overlap = text[-overlap_size:]
    
#     sentence_start = overlap.find('. ')
#     if sentence_start != -1 and sentence_start < overlap_size * 0.5:
#         overlap = overlap[sentence_start + 2:]
    
#     return overlap

# all_chunks = []
# MAX_SIZE = 950

# # Process each page
# for page_num in range(total_pages):
#     print(f"\n   Page {page_num + 1}/{total_pages}")
#     print(f"      Extracting text (raw)...")
    
#     page = doc[page_num]
#     text = page.get_text("text", sort=True)
    
#     if not text.strip():
#         print(f"       ✅ 0 text chunks (empty page)")
#         continue
    
#     # Chunking with overlap
#     page_chunks = []
#     chunk_idx = 0
    
#     paragraphs = [p.strip() for p in text.split('\n\n') if p.strip() and len(p.strip()) > 10]
#     current_chunk = ""
#     overlap_text = ""
    
#     for para in paragraphs:
#         if len(para) > MAX_SIZE:
#             if current_chunk.strip():
#                 page_chunks.append({
#                     'page': page_num + 1,
#                     'chunk_idx': chunk_idx,
#                     'text': current_chunk.strip()
#                 })
#                 chunk_idx += 1
#                 overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
#                 current_chunk = overlap_text
            
#             sentences = [s.strip() + '.' for s in para.split('.') if s.strip()]
#             for sent in sentences:
#                 if len(current_chunk) + len(sent) > MAX_SIZE:
#                     if current_chunk.strip():
#                         page_chunks.append({
#                             'page': page_num + 1,
#                             'chunk_idx': chunk_idx,
#                             'text': current_chunk.strip()
#                         })
#                         chunk_idx += 1
#                         overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
#                     current_chunk = overlap_text + sent + " "
#                 else:
#                     current_chunk += sent + " "
        
#         elif len(current_chunk) + len(para) > MAX_SIZE:
#             if current_chunk.strip():
#                 page_chunks.append({
#                     'page': page_num + 1,
#                     'chunk_idx': chunk_idx,
#                     'text': current_chunk.strip()
#                 })
#                 chunk_idx += 1
#                 overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
#             current_chunk = overlap_text + para + "\n\n"
#         else:
#             current_chunk += para + "\n\n"
    
#     # Save last chunk
#     if current_chunk.strip():
#         if len(current_chunk) > 1000:
#             current_chunk = current_chunk[:1000]
#         page_chunks.append({
#             'page': page_num + 1,
#             'chunk_idx': chunk_idx,
#             'text': current_chunk.strip()
#         })
    
#     all_chunks.extend(page_chunks)
#     print(f"       ✅ {len(page_chunks)} text chunks")

# doc.close()

# print(f"\n{'='*70}")
# print(f"✅ EXTRACTION COMPLETE: {len(all_chunks)} total chunks")
# print(f"{'='*70}")

# # Print all chunks WITH OVERLAP HIGHLIGHTING
# print(f"\n\n📄 ALL EXTRACTED CHUNKS (with overlap visualization):")
# print("="*70)

# for i, chunk in enumerate(all_chunks):
#     print(f"\n{'='*70}")
#     print(f"CHUNK {i+1} | Page {chunk['page']} | Chunk Index {chunk['chunk_idx']}")
#     print(f"Length: {len(chunk['text'])} chars")
    
#     # Highlight overlap with previous chunk
#     if i > 0:
#         prev_chunk = all_chunks[i-1]['text']
#         curr_chunk = chunk['text']
        
#         # Find overlap between previous and current
#         overlap_found = None
#         for overlap_len in range(min(len(prev_chunk), len(curr_chunk)), 20, -1):
#             if prev_chunk[-overlap_len:] == curr_chunk[:overlap_len]:
#                 overlap_found = overlap_len
#                 break
        
#         if overlap_found and overlap_found > 30:
#             print(f"🔄 OVERLAP DETECTED: {overlap_found} chars")
#             print(f"   Previous chunk ends with: ...{prev_chunk[-100:]}")
#             print(f"   This chunk starts with: {curr_chunk[:100]}...")
    
#     print(f"{'='*70}")
#     print(chunk['text'])

# print(f"\n\n{'='*70}")
# print(f"📊 SUMMARY")
# print(f"{'='*70}")
# print(f"Total chunks extracted: {len(all_chunks)}")
# print(f"Pages processed: {total_pages}")
# print(f"Overlap setting: {OVERLAP_SIZE} chars")
# print(f"Average chunk size: {sum(len(c['text']) for c in all_chunks) / len(all_chunks):.0f} chars")

In [10]:
#!pip install docling

In [11]:
# TEST DOCLING EXTRACTION WITH OVERLAP - Single Cell
from docling.document_converter import DocumentConverter
from pathlib import Path
import yaml

# Load config
with open("config/config.yaml", 'r') as f:
    config = yaml.safe_load(f)

max_pages = config['processing']['max_test_pages']
OVERLAP_SIZE = 100  # Fixed overlap size

# Pick a PDF to test
pdf_path = Path("test_dataset/markdown_extraction/1-s2.0-S0720048X23000712-main.pdf")

print("="*70)
print(f"📄 EXTRACTING WITH DOCLING: {pdf_path.name}")
print(f"   Pages to process: {max_pages}")
print(f"   Overlap: {OVERLAP_SIZE} chars")
print("="*70)

# Initialize Docling converter
converter = DocumentConverter()

# Convert PDF
print("\n🔄 Converting PDF with Docling...")
result = converter.convert(str(pdf_path))

# Get markdown output
markdown_text = result.document.export_to_markdown()

print(f"✅ Conversion complete!")
print(f"📊 Total markdown length: {len(markdown_text)} chars")
print("="*70)

# NOW CHUNK THE MARKDOWN WITH OVERLAP
print(f"\n\n📝 CHUNKING MARKDOWN OUTPUT WITH {OVERLAP_SIZE} CHAR OVERLAP...")
print("="*70)

def get_overlap(text: str, overlap_size: int) -> str:
    """Get the last N characters from text, trying to break at sentence boundary."""
    if len(text) <= overlap_size:
        return text
    
    overlap = text[-overlap_size:]
    sentence_start = overlap.find('. ')
    if sentence_start != -1 and sentence_start < overlap_size * 0.5:
        overlap = overlap[sentence_start + 2:]
    
    return overlap

all_chunks = []
MAX_SIZE = 950
chunk_idx = 0

# Split by paragraphs (markdown double newlines)
paragraphs = [p.strip() for p in markdown_text.split('\n\n') if p.strip() and len(p.strip()) > 10]

print(f"Found {len(paragraphs)} paragraphs to chunk\n")

current_chunk = ""
overlap_text = ""

for para in paragraphs:
    # If single para too long, split by sentences
    if len(para) > MAX_SIZE:
        if current_chunk.strip():
            all_chunks.append({
                'chunk_idx': chunk_idx,
                'text': current_chunk.strip(),
                'length': len(current_chunk.strip())
            })
            chunk_idx += 1
            overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
            current_chunk = overlap_text
        
        # Split by sentences
        sentences = [s.strip() + '.' for s in para.split('.') if s.strip()]
        for sent in sentences:
            if len(current_chunk) + len(sent) > MAX_SIZE:
                if current_chunk.strip():
                    all_chunks.append({
                        'chunk_idx': chunk_idx,
                        'text': current_chunk.strip(),
                        'length': len(current_chunk.strip())
                    })
                    chunk_idx += 1
                    overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
                current_chunk = overlap_text + sent + " "
            else:
                current_chunk += sent + " "
    
    # Check if adding para would exceed limit
    elif len(current_chunk) + len(para) > MAX_SIZE:
        if current_chunk.strip():
            all_chunks.append({
                'chunk_idx': chunk_idx,
                'text': current_chunk.strip(),
                'length': len(current_chunk.strip())
            })
            chunk_idx += 1
            overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
        current_chunk = overlap_text + para + "\n\n"
    else:
        current_chunk += para + "\n\n"

# Save last chunk
if current_chunk.strip():
    if len(current_chunk) > 1000:  # Safety truncate
        current_chunk = current_chunk[:1000]
    all_chunks.append({
        'chunk_idx': chunk_idx,
        'text': current_chunk.strip(),
        'length': len(current_chunk.strip())
    })

print(f"\n{'='*70}")
print(f"✅ CHUNKING COMPLETE: {len(all_chunks)} total chunks")
print(f"{'='*70}")

# PRINT ALL CHUNKS WITH OVERLAP VISUALIZATION
print(f"\n\n📄 ALL EXTRACTED CHUNKS (with overlap visualization):")
print("="*70)

for i, chunk in enumerate(all_chunks):
    print(f"\n{'='*70}")
    print(f"CHUNK {i+1} | Chunk Index {chunk['chunk_idx']}")
    print(f"Length: {chunk['length']} chars")
    
    # Highlight overlap with previous chunk
    if i > 0:
        prev_chunk = all_chunks[i-1]['text']
        curr_chunk = chunk['text']
        
        # Find overlap between previous and current
        overlap_found = None
        for overlap_len in range(min(len(prev_chunk), len(curr_chunk)), 20, -1):
            if prev_chunk[-overlap_len:] == curr_chunk[:overlap_len]:
                overlap_found = overlap_len
                break
        
        if overlap_found and overlap_found > 30:
            print(f"🔄 OVERLAP DETECTED: {overlap_found} chars")
            print(f"   📌 Previous chunk ends with: ...{prev_chunk[-100:]}")
            print(f"   📌 This chunk starts with: {curr_chunk[:100]}...")
    
    print(f"{'='*70}")
    print(chunk['text'])

print(f"\n\n{'='*70}")
print(f"📊 SUMMARY")
print(f"{'='*70}")
print(f"Total chunks extracted: {len(all_chunks)}")
print(f"Overlap setting: {OVERLAP_SIZE} chars")
print(f"Average chunk size: {sum(c['length'] for c in all_chunks) / len(all_chunks):.0f} chars")
print(f"Min chunk size: {min(c['length'] for c in all_chunks)} chars")
print(f"Max chunk size: {max(c['length'] for c in all_chunks)} chars")

2025-10-16 15:19:24,618 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-16 15:19:24,645 - INFO - Going to convert document batch...
2025-10-16 15:19:24,646 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 4f2edc0f7d9bb60b38ebfecf9a2609f5
2025-10-16 15:19:24,653 - INFO - Loading plugin 'docling_defaults'
2025-10-16 15:19:24,654 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-16 15:19:24,660 - INFO - Loading plugin 'docling_defaults'
2025-10-16 15:19:24,662 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']


📄 EXTRACTING WITH DOCLING: 1-s2.0-S0720048X23000712-main.pdf
   Pages to process: 25
   Overlap: 100 chars

🔄 Converting PDF with Docling...


2025-10-16 15:19:28,056 - INFO - Auto OCR model selected ocrmac.
2025-10-16 15:19:28,079 - INFO - Accelerator device: 'mps'
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
2025-10-16 15:19:28,775 - WARNING - Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
2025-10-16 15:19:37,316 - WARNING - Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTT

✅ Conversion complete!
📊 Total markdown length: 87574 chars


📝 CHUNKING MARKDOWN OUTPUT WITH 100 CHAR OVERLAP...
Found 119 paragraphs to chunk


✅ CHUNKING COMPLETE: 130 total chunks


📄 ALL EXTRACTED CHUNKS (with overlap visualization):

CHUNK 1 | Chunk Index 0
Length: 500 chars
<!-- image -->

Contents lists available at ScienceDirect

## European Journal of Radiology

journal homepage: www.elsevier.com/locate/ejrad

## Prostate MRI and image Quality: It is time to take stock

Yue Lin a , Enis C. Yilmaz a , Mason J. Belue a , Baris Turkbey a,*

- a Molecular Imaging Branch, National Cancer Institute, National Institutes of Health, Bethesda, MD, United States

A R T I C L E  I N F O

Prostatic Neoplasms

Magnetic Resonance Imaging

Diagnostic Imaging

## 1. Introduction

CHUNK 2 | Chunk Index 1
Length: 844 chars
🔄 OVERLAP DETECTED: 96 chars
   📌 Previous chunk ends with: ... E  I N F O

Prostatic Neoplasms

Magnetic Resonance Imaging

Diagnostic Imaging

## 1. Introduction
   📌 This 

In [17]:
#!pip install markitdown

In [18]:
# TEST MARKITDOWN EXTRACTION WITH OVERLAP - Single Cell
from markitdown import MarkItDown
from pathlib import Path
import yaml

# Load config
with open("config/config.yaml", 'r') as f:
    config = yaml.safe_load(f)

max_pages = config['processing']['max_test_pages']
OVERLAP_SIZE = 100  # Fixed overlap size

# Pick a PDF to test
pdf_path = Path("test_dataset/markdown_extraction/1-s2.0-S0720048X23000712-main.pdf")

print("="*70)
print(f"📄 EXTRACTING WITH MARKITDOWN: {pdf_path.name}")
print(f"   Overlap: {OVERLAP_SIZE} chars")
print("="*70)

# Initialize MarkItDown
md = MarkItDown()

# Convert PDF
print("\n🔄 Converting PDF with MarkItDown...")
result = md.convert(str(pdf_path))

# Get markdown output
markdown_text = result.text_content

print(f"✅ Conversion complete!")
print(f"📊 Total markdown length: {len(markdown_text)} chars")
print("="*70)

# NOW CHUNK THE MARKDOWN WITH OVERLAP
print(f"\n\n📝 CHUNKING MARKDOWN OUTPUT WITH {OVERLAP_SIZE} CHAR OVERLAP...")
print("="*70)

def get_overlap(text: str, overlap_size: int) -> str:
    """Get the last N characters from text, trying to break at sentence boundary."""
    if len(text) <= overlap_size:
        return text
    
    overlap = text[-overlap_size:]
    sentence_start = overlap.find('. ')
    if sentence_start != -1 and sentence_start < overlap_size * 0.5:
        overlap = overlap[sentence_start + 2:]
    
    return overlap

all_chunks = []
MAX_SIZE = 950
chunk_idx = 0

# Split by paragraphs (markdown double newlines)
paragraphs = [p.strip() for p in markdown_text.split('\n\n') if p.strip() and len(p.strip()) > 10]

print(f"Found {len(paragraphs)} paragraphs to chunk\n")

current_chunk = ""
overlap_text = ""

for para in paragraphs:
    # If single para too long, split by sentences
    if len(para) > MAX_SIZE:
        if current_chunk.strip():
            all_chunks.append({
                'chunk_idx': chunk_idx,
                'text': current_chunk.strip(),
                'length': len(current_chunk.strip())
            })
            chunk_idx += 1
            overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
            current_chunk = overlap_text
        
        # Split by sentences
        sentences = [s.strip() + '.' for s in para.split('.') if s.strip()]
        for sent in sentences:
            if len(current_chunk) + len(sent) > MAX_SIZE:
                if current_chunk.strip():
                    all_chunks.append({
                        'chunk_idx': chunk_idx,
                        'text': current_chunk.strip(),
                        'length': len(current_chunk.strip())
                    })
                    chunk_idx += 1
                    overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
                current_chunk = overlap_text + sent + " "
            else:
                current_chunk += sent + " "
    
    # Check if adding para would exceed limit
    elif len(current_chunk) + len(para) > MAX_SIZE:
        if current_chunk.strip():
            all_chunks.append({
                'chunk_idx': chunk_idx,
                'text': current_chunk.strip(),
                'length': len(current_chunk.strip())
            })
            chunk_idx += 1
            overlap_text = get_overlap(current_chunk, OVERLAP_SIZE)
        current_chunk = overlap_text + para + "\n\n"
    else:
        current_chunk += para + "\n\n"

# Save last chunk
if current_chunk.strip():
    if len(current_chunk) > 1000:  # Safety truncate
        current_chunk = current_chunk[:1000]
    all_chunks.append({
        'chunk_idx': chunk_idx,
        'text': current_chunk.strip(),
        'length': len(current_chunk.strip())
    })

print(f"\n{'='*70}")
print(f"✅ CHUNKING COMPLETE: {len(all_chunks)} total chunks")
print(f"{'='*70}")

# PRINT ALL CHUNKS WITH OVERLAP VISUALIZATION
print(f"\n\n📄 ALL EXTRACTED CHUNKS (with overlap visualization):")
print("="*70)

for i, chunk in enumerate(all_chunks):
    print(f"\n{'='*70}")
    print(f"CHUNK {i+1} | Chunk Index {chunk['chunk_idx']}")
    print(f"Length: {chunk['length']} chars")
    
    # Highlight overlap with previous chunk
    if i > 0:
        prev_chunk = all_chunks[i-1]['text']
        curr_chunk = chunk['text']
        
        # Find overlap between previous and current
        overlap_found = None
        for overlap_len in range(min(len(prev_chunk), len(curr_chunk)), 20, -1):
            if prev_chunk[-overlap_len:] == curr_chunk[:overlap_len]:
                overlap_found = overlap_len
                break
        
        if overlap_found and overlap_found > 30:
            print(f"🔄 OVERLAP DETECTED: {overlap_found} chars")
            print(f"   📌 Previous chunk ends with: ...{prev_chunk[-100:]}")
            print(f"   📌 This chunk starts with: {curr_chunk[:100]}...")
    
    print(f"{'='*70}")
    print(chunk['text'])

print(f"\n\n{'='*70}")
print(f"📊 SUMMARY")
print(f"{'='*70}")
print(f"Total chunks extracted: {len(all_chunks)}")
print(f"Overlap setting: {OVERLAP_SIZE} chars")
print(f"Average chunk size: {sum(c['length'] for c in all_chunks) / len(all_chunks):.0f} chars")
print(f"Min chunk size: {min(c['length'] for c in all_chunks)} chars")
print(f"Max chunk size: {max(c['length'] for c in all_chunks)} chars")

📄 EXTRACTING WITH MARKITDOWN: 1-s2.0-S0720048X23000712-main.pdf
   Overlap: 100 chars

🔄 Converting PDF with MarkItDown...
✅ Conversion complete!
📊 Total markdown length: 76189 chars


📝 CHUNKING MARKDOWN OUTPUT WITH 100 CHAR OVERLAP...
Found 431 paragraphs to chunk


✅ CHUNKING COMPLETE: 109 total chunks


📄 ALL EXTRACTED CHUNKS (with overlap visualization):

CHUNK 1 | Chunk Index 0
Length: 478 chars
Contents lists available at ScienceDirect

European Journal of Radiology

journal homepage: www.elsevier.com/locate/ejrad

Prostate MRI and image Quality: It is time to take stock

Yue Lin a, Enis C. Yilmaz a, Mason J. Belue a, Baris Turkbey a, *
a Molecular Imaging Branch, National Cancer Institute, National Institutes of Health, Bethesda, MD, United States

A R T I C L E  I N F O

A B S T R A C T

Keywords:
Prostatic Neoplasms
Magnetic Resonance Imaging
Diagnostic Imaging

CHUNK 2 | Chunk Index 1
Length: 891 chars
🔄 OVERLAP DETECTED: 97 chars
   📌 Previous chunk ends with: ... N F O

A 